In [44]:
import pandas as pd 
import re
import string
import findspark 
findspark.init()
import pyspark as ps
from pyspark.sql.types import *
import re
import string
from datetime import datetime
from pyspark.sql.functions import array_contains
from pyspark.sql.functions import col
from pyspark.sql.functions import udf
from pyspark.ml.feature import Tokenizer, HashingTF, IDF, CountVectorizer

In [6]:
def txt_to_feature(text):
    words = re.sub("[^a-zA-Z]", " ", text).lower().split()
    clean_words = [ps.stem(w) for w in words if not w in stops]
    return( " ".join(clean_words))

In [8]:
spark = ps.sql.SparkSession.builder \
            .master("local[4]") \
            .appName('Model Training') \
            .getOrCreate()


In [9]:
schema = StructType([
        StructField('Index' , IntegerType(), True),
        StructField('ItemID' , IntegerType(), True),
        StructField('Sentiment' , IntegerType(), True),
        StructField('SentimentSource' , StringType(), True),
        StructField('SentimentText' , StringType(), True),
    ])

In [10]:
filename = "../data/cleaned-Sentiment-Analysis-Dataset.csv"
users = spark.read.csv(filename, header=True, schema=schema)

In [11]:
users.printSchema()

root
 |-- Index: integer (nullable = true)
 |-- ItemID: integer (nullable = true)
 |-- Sentiment: integer (nullable = true)
 |-- SentimentSource: string (nullable = true)
 |-- SentimentText: string (nullable = true)



In [12]:
users.show(5)

+-----+------+---------+---------------+--------------------+
|Index|ItemID|Sentiment|SentimentSource|       SentimentText|
+-----+------+---------+---------------+--------------------+
|    0|     1|        0|   Sentiment140|                 ...|
|    1|     2|        0|   Sentiment140|                 ...|
|    2|     3|        1|   Sentiment140|              omg...|
|    3|     4|        0|   Sentiment140|          .. Omga...|
|    4|     5|        0|   Sentiment140|         i think ...|
+-----+------+---------+---------------+--------------------+
only showing top 5 rows



In [30]:
def preprocess(text):
    words = re.sub("[^a-zA-Z]", " ", text).lower().split()
    return words

In [31]:
pp_udf = udf(preprocess, ArrayType(StringType()))

In [32]:
df = users.withColumn('Words', pp_udf(users.SentimentText))

In [33]:
df.printSchema()

root
 |-- Index: integer (nullable = true)
 |-- ItemID: integer (nullable = true)
 |-- Sentiment: integer (nullable = true)
 |-- SentimentSource: string (nullable = true)
 |-- SentimentText: string (nullable = true)
 |-- Words: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [122]:
from pyspark.ml.feature import StopWordsRemover
remover = StopWordsRemover(inputCol="Words", outputCol="filtered")
remover.transform(df).show()

IllegalArgumentException: 'requirement failed: Column filtered already exists.'

In [97]:
removed = remover.transform(df)
removed.show()

+-----+------+---------+---------------+--------------------+--------------------+--------------------+
|Index|ItemID|Sentiment|SentimentSource|       SentimentText|               Words|            filtered|
+-----+------+---------+---------------+--------------------+--------------------+--------------------+
|    0|     1|        0|   Sentiment140|                 ...|[is, so, sad, for...|  [sad, apl, friend]|
|    1|     2|        0|   Sentiment140|                 ...|[i, missed, the, ...|[missed, new, moo...|
|    2|     3|        1|   Sentiment140|              omg...|[omg, its, alread...|      [omg, already]|
|    3|     4|        0|   Sentiment140|          .. Omga...|[omgaga, im, sooo...|[omgaga, im, sooo...|
|    4|     5|        0|   Sentiment140|         i think ...|[i, think, mi, bf...|[think, mi, bf, c...|
|    5|     6|        0|   Sentiment140|         or i jus...|[or, i, just, wor...|       [worry, much]|
|    6|     7|        1|   Sentiment140|       Juuuuuuuuu...|[ju

In [119]:
new_df = removed.select(col('Sentiment').alias('label'), col('SentimentText'), col('Words'), col('filtered'))

In [103]:
new_df.show()

+-----+--------------------+--------------------+--------------------+
|label|       SentimentText|               Words|            filtered|
+-----+--------------------+--------------------+--------------------+
|    0|                 ...|[is, so, sad, for...|  [sad, apl, friend]|
|    0|                 ...|[i, missed, the, ...|[missed, new, moo...|
|    1|              omg...|[omg, its, alread...|      [omg, already]|
|    0|          .. Omga...|[omgaga, im, sooo...|[omgaga, im, sooo...|
|    0|         i think ...|[i, think, mi, bf...|[think, mi, bf, c...|
|    0|         or i jus...|[or, i, just, wor...|       [worry, much]|
|    1|       Juuuuuuuuu...|[juuuuuuuuuuuuuuu...|[juuuuuuuuuuuuuuu...|
|    0|       Sunny Agai...|[sunny, again, wo...|[sunny, work, tom...|
|    1|      handed in m...|[handed, in, my, ...|[handed, uniform,...|
|    1|      hmmmm.... i...|[hmmmm, i, wonder...|[hmmmm, wonder, n...|
|    0|      I must thin...|[i, must, think, ...|[must, think, pos...|
|    1

In [120]:
hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=200)
featurizedData = hashingTF.transform(new_df)


In [115]:
featurizedData.cache()

DataFrame[label: int, SentimentText: string, Words: array<string>, filtered: array<string>, rawFeatures: vector]

In [121]:
featurizedData.show()

Py4JJavaError: An error occurred while calling o1852.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 63.0 failed 1 times, most recent failure: Lost task 0.0 in stage 63.0 (TID 107, localhost, executor driver): java.lang.NumberFormatException: For input string: "Kaggle"
	at java.lang.NumberFormatException.forInputString(NumberFormatException.java:65)
	at java.lang.Integer.parseInt(Integer.java:580)
	at java.lang.Integer.parseInt(Integer.java:615)
	at scala.collection.immutable.StringLike$class.toInt(StringLike.scala:272)
	at scala.collection.immutable.StringOps.toInt(StringOps.scala:29)
	at org.apache.spark.sql.execution.datasources.csv.CSVTypeCast$.castTo(CSVInferSchema.scala:250)
	at org.apache.spark.sql.execution.datasources.csv.CSVRelation$$anonfun$csvParser$3.apply(CSVRelation.scala:125)
	at org.apache.spark.sql.execution.datasources.csv.CSVRelation$$anonfun$csvParser$3.apply(CSVRelation.scala:94)
	at org.apache.spark.sql.execution.d

In [112]:
idf = IDF(inputCol="rawFeatures", outputCol="features")
model = idf.fit(featurizedData)
result = model.transform(featurizedData)

Py4JJavaError: An error occurred while calling o1663.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 60.0 failed 1 times, most recent failure: Lost task 0.0 in stage 60.0 (TID 101, localhost, executor driver): java.lang.NumberFormatException: For input string: "Kaggle"
	at java.lang.NumberFormatException.forInputString(NumberFormatException.java:65)
	at java.lang.Integer.parseInt(Integer.java:580)
	at java.lang.Integer.parseInt(Integer.java:615)
	at scala.collection.immutable.StringLike$class.toInt(StringLike.scala:272)
	at scala.collection.immutable.StringOps.toInt(StringOps.scala:29)
	at org.apache.spark.sql.execution.datasources.csv.CSVTypeCast$.castTo(CSVInferSchema.scala:250)
	at org.apache.spark.sql.execution.datasources.csv.CSVRelation$$anonfun$csvParser$3.apply(CSVRelation.scala:125)
	at org.apache.spark.sql.execution.datasources.csv.CSVRelation$$anonfun$csvParser$3.apply(CSVRelation.scala:94)
	at org.apache.spark.sql.execution.datasour

In [56]:
sentenceData = spark.createDataFrame([
    (0.0, "Hi I heard about Spark"),
    (0.0, "I wish Java could use case classes"),
    (1.0, "Logistic regression models are neat")
], ["label", "sentence"])

tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
wordsData = tokenizer.transform(sentenceData)

hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)
featurizedData = hashingTF.transform(wordsData)

# alternatively, CountVectorizer can also be used to get term frequency vectors

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

rescaledData.select("label", "features").show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(20,[0,5,9,17],[0...|
|  0.0|(20,[2,7,9,13,15]...|
|  1.0|(20,[4,6,13,15,18...|
+-----+--------------------+



In [80]:
featurizedData.show()

+-----+------+---------+---------------+--------------------+--------------------+--------------------+--------------------+-----+
|Index|ItemID|Sentiment|SentimentSource|       SentimentText|               Words|            filtered|         rawFeatures|label|
+-----+------+---------+---------------+--------------------+--------------------+--------------------+--------------------+-----+
|    0|     1|        0|   Sentiment140|                 ...|[is, so, sad, for...|  [sad, apl, friend]|(200,[38,60,193],...|    0|
|    1|     2|        0|   Sentiment140|                 ...|[i, missed, the, ...|[missed, new, moo...|(200,[15,25,81,19...|    0|
|    2|     3|        1|   Sentiment140|              omg...|[omg, its, alread...|      [omg, already]|(200,[57,184],[1....|    1|
|    3|     4|        0|   Sentiment140|          .. Omga...|[omgaga, im, sooo...|[omgaga, im, sooo...|(200,[18,56,73,82...|    0|
|    4|     5|        0|   Sentiment140|         i think ...|[i, think, mi, bf...|[

In [49]:
idf = IDF(minDocFreq=3, inputCol="filtered", outputCol="features")
idfModel = idf.fit(clean_df)
#fitted = idfModel.transform(clean_df).show(5)

IllegalArgumentException: 'requirement failed: Column filtered must be of type org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7 but was actually ArrayType(StringType,true).'